In [ ]:
import cv2
import numpy as np

video_capture_0=cv2.VideoCapture(0)
video_capture_1=cv2.VideoCapture(1)
while True:
    ret0,frame0 =video_capture_0.read()
    ret1,frame1=video_capture_1.read()
    MIN_MATCH_COUNT = 10

    img1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
# Initiate SIFT detector
    sift = cv2.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

# store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()

        h,w = img1.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)

        img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
        matchesMask = None
    warped_image = cv2.warpPerspective(img1, M, (w, h))
    warped_image = cv2.cvtColor(warped_image, cv2.COLOR_GRAY2RGB)

# Blend the warped image with image2
    blended_image = cv2.addWeighted(frame1, 0.6, warped_image, 0.4,0)
    blended_image = cv2.resize(blended_image,(1000,720))
    cv2.imshow('res',blended_image)
   
    if cv2.waitKey(2) & 0xff == ord('q'):
        break


video_capture_0.release()
video_capture_1.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import imutils

img_rgb = cv2.imread("suspension-insulator-.jpg")
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread(r"C:\Users\adity\Downloads\WhatsApp Image 2023-05-30 at 5.02.51 PM.jpeg",0)
# cv2.imshow('img',template)

cv2.imwrite('resized.jpg',template)
template = cv2.resize(template,(1000,1000))
# for scale in np.linspace(0.025, 0.1, 40):
resized_template = imutils.resize(template, width = 400)
    
res = cv2.matchTemplate(img_gray, resized_template, cv2.TM_SQDIFF)
min_val, _, min_loc, _ = cv2.minMaxLoc(res)
# if best_match is None or min_val <= best_match[0]:
# ideal_scale=scale 
h, w = resized_template.shape[::]
best_match = [min_val, min_loc]
        
        
# print("Ideal template image size is : ",ideal_scale)
top_left = best_match[1]  
bottom_right = (top_left[0] + w, top_left[1] + h)
cv2.rectangle(img_rgb, top_left, bottom_right, (0, 0,255), 2) 
cv2.imwrite('matched_resized3.jpg', img_rgb)

In [ ]:
cv2.imshow('img',template)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import tensorflow
import pixellib
from pixellib.tune_bg import alter_bg

change_bg = alter_bg(model_type = "pb")
change_bg.load_pascalvoc_model("xception_pascalvoc.pb")
change_bg.blur_bg("image.jpg", extreme = True, detect = "person", output_image_name="blur_img.jpg")

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

MIN_MATCH_COUNT = 30
image1 = cv2.imread(r'C:\Users\adity\Downloads\img2.jpeg')
image2 = cv2.imread(r'C:\Users\adity\Downloads\img1.jpeg')
image2 = cv2.resize(image2,(500,500))
image1 = cv2.resize(image1,(500,500))

img1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
cv2.imshow('image1',image1)
cv2.imshow('image2',image2)
# Initiate SIFT detector
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1,des2,k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

else:
    print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None
warped_image = cv2.warpPerspective(img1, M, (w, h))


warped_image = cv2.cvtColor(warped_image, cv2.COLOR_GRAY2RGB)
warped_image = cv2.resize(warped_image,(image2.shape[1],image2.shape[0]))
# Blend the warped image with image2
blended_image = cv2.addWeighted(image2, 0.9, warped_image, 0.1,0)
# blended_image = cv2.resize(blended_image,(1000,720))
# Display the result
cv2.imshow('Blended Image', blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# draw_params = dict(matchColor = (0,255,0), # draw matches in green color
#                    singlePointColor = None,
#                    matchesMask = matchesMask, # draw only inliers
#                    flags = 2)
# img3 = cv2.add(img2,img1)
# img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

# plt.imshow(img3,'gray'),plt.show()

In [ ]:
import cv2
image2 = cv2.imread(r"C:\Users\adity\Downloads\signcropped.jpeg")
cv2.imshow('image2',image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
len(good)

In [ ]:
a = img1.shape
b = img2.shape
print(a,b)

In [ ]:
from PIL import Image
import cv2
import numpy

imag = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg")

# coordinates of the pixel
X, Y = 0, 0
# Get RGB
pixelRGB = imag.getpixel((X, Y))
R, G, B = pixelRGB
brightness = sum([R, G, B]) / 3  ##0 is dark (black) and 255 is bright (white)
print(brightness)

In [ ]:
!pip install getpixel

In [ ]:
from PIL import Image, ImageEnhance

# Load the image
# image = Image.open("path/to/your/image.jpg")
image = Image.open(r"C:\Users\adity\Downloads\img2.jpeg")
# Adjust brightness
enhancer = ImageEnhance.Brightness(image)
brightened_image = enhancer.enhance(.67)  # Increase brightness by a factor of 1.5

# Adjust contrast
enhancer = ImageEnhance.Contrast(brightened_image)
final_image = enhancer.enhance(1.15)  # Increase contrast by a factor of 1.2

# Save the adjusted image
final_image.save('img5.jpg')


In [ ]:
import numpy as np
from PIL import Image

# Load the image
image = Image.open(r"img5.jpg")

# Convert the image to grayscale
grayscale_image = image.convert("L")

# Convert the grayscale image to a NumPy array
image_array = np.array(grayscale_image)

# Calculate the average pixel value (brightness)
brightness = np.mean(image_array)

# Print the brightness value
print("Brightness:", brightness)

std_dev = np.std(image_array)

# Calculate the contrast as the standard deviation divided by the mean
contrast = std_dev / brightness

# Print the contrast value
print("Contrast:", contrast)



In [ ]:
image = Image.open(r"C:\Users\adity\Downloads\img1.jpeg")

# Convert the image to grayscale
grayscale_image = image.convert("L")

# Convert the grayscale image to a NumPy array
image_array = np.array(grayscale_image)

# Calculate the average pixel value (brightness)
brightness = np.mean(image_array)

# Print the brightness value
print("Brightness:", brightness)

std_dev = np.std(image_array)

# Calculate the contrast as the standard deviation divided by the mean
contrast = std_dev / brightness

# Print the contrast value
print("Contrast:", contrast)

In [ ]:
import cv2
import numpy as np

# Load the two images
image1 = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg")
image2 = cv2.imread(r"C:\Users\adity\Downloads\img1.jpeg")

# Preprocess the images (resize, convert to grayscale, etc.)
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Feature extraction (SIFT)
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(image1_gray, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2_gray, None)

# Feature matching (Brute-force)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)

# Sort matches by distance
matches = sorted(matches, key=lambda x: x.distance)

# Select top n matches
n = 50
matches = matches[:n]

# Extract matching keypoints
points1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
points2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# Compute homography matrix
M, mask = cv2.findHomography(points1, points2, cv2.RANSAC, 5.0)

# Warp image1 to align with image2
result_size = (image2.shape[1], image2.shape[0])
aligned_image1 = cv2.warpPerspective(image1, M, result_size)

# Create a mask for the common portion
mask = np.zeros_like(image2_gray)
cv2.fillPoly(mask, [np.int32(points2)], (255, 255, 255))

# Convert the mask to 3-channel
mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
mask_3ch = cv2.normalize(mask_3ch, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
# Apply the mask to the aligned image
masked_image1 = cv2.bitwise_and(aligned_image1, mask_3ch)
masked_image1 = cv2.normalize(masked_image1, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
# Perform the superimposition
result = cv2.addWeighted(image2, 0.9, masked_image1, 0.1, 0)
cv2.imwrite('superimpose.jpg',result)
# Visualize the superimposed image
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
pip install opencv-contrib-python

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

im1 = cv2.imread(r"C:\Users\adity\Downloads\img1.jpeg")          # Image that needs to be registered.
im2 = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg") # trainImage

img1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

# Initiate ORB detector
orb = cv2.ORB_create(100)  #Registration works with at least 50 points

# find the keypoints and descriptors with orb
kp1, des1 = orb.detectAndCompute(img1, None)  #kp1 --> list of keypoints
kp2, des2 = orb.detectAndCompute(img2, None)

#Brute-Force matcher takes the descriptor of one feature in first set and is 
#matched with all other features in second set using some distance calculation.
# create Matcher object

matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

# Match descriptors.
matches = matcher.match(des1, des2, None)  #Creates a list of all matches, just like keypoints

# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

#Like we used cv2.drawKeypoints() to draw keypoints, 
#cv2.drawMatches() helps us to draw the matches. 
#https://docs.opencv.org/3.0-beta/modules/features2d/doc/drawing_function_of_keypoints_and_matches.html
# Draw first 10 matches.
img3 = cv2.drawMatches(im1,kp1, im2, kp2, matches[:10], None)

cv2.imshow("Matches image", img3)
cv2.waitKey(0)

#Now let us use these key points to register two images. 
#Can be used for distortion correction or alignment
#For this task we will use homography. 
# https://docs.opencv.org/3.4.1/d9/dab/tutorial_homography.html

# Extract location of good matches.
# For this we will use RANSAC.
#RANSAC is abbreviation of RANdom SAmple Consensus, 
#in summary it can be considered as outlier rejection method for keypoints.
#http://eric-yuan.me/ransac/
#RANSAC needs all key points indexed, first set indexed to queryIdx
#Second set to #trainIdx. 

points1 = np.zeros((len(matches), 2), dtype=np.float32)  #Prints empty array of size equal to (matches, 2)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
   points1[i, :] = kp1[match.queryIdx].pt    #gives index of the descriptor in the list of query descriptors
   points2[i, :] = kp2[match.trainIdx].pt    #gives index of the descriptor in the list of train descriptors

#Now we have all good keypoints so we are ready for homography.   
# Find homography
#https://en.wikipedia.org/wiki/Homography_(computer_vision)
  
h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
height, width, channels = im2.shape
im1Reg = cv2.warpPerspective(im1, h, (width, height))  #Applies a perspective transformation to an image.

cv2.imshow("Registered image", im1Reg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Load the two images
image1 = cv2.imread(r"C:\Users\adity\Downloads\img1.jpeg")
image2 = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg")

# Perform histogram matching to make the images visually consistent
matched_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
matched_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)
matched_image2[:, :, 0] = cv2.normalize(matched_image2[:, :, 0], matched_image1[:, :, 0].min(), matched_image1[:, :, 0].max(), norm_type=cv2.NORM_MINMAX)
matched_image2 = cv2.cvtColor(matched_image2, cv2.COLOR_Lab2BGR)

# Preprocess the images (resize, convert to grayscale, etc.)
image1_gray = cv2.cvtColor(matched_image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(matched_image2, cv2.COLOR_BGR2GRAY)

# Feature extraction (SIFT)
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(image1_gray, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2_gray, None)

# Feature matching (Brute-force)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)

# Sort matches by distance
matches = sorted(matches, key=lambda x: x.distance)

# Select top n matches
n = 50
matches = matches[:n]

# Extract matching keypoints
points1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
points2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# Compute homography matrix
M, mask = cv2.findHomography(points1, points2, cv2.RANSAC, 5.0)

# Warp image1 to align with image2
result_size = (image2.shape[1], image2.shape[0])
aligned_image1 = cv2.warpPerspective(matched_image1, M, result_size)

# Create a mask for the common portion
mask = np.zeros_like(image2_gray)
cv2.fillPoly(mask, [np.int32(points2)], (255, 255, 255))

# Convert the mask to 3-channel
mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

# Apply the mask to the aligned image
masked_image1 = cv2.bitwise_and(aligned_image1, mask_3ch)

# Perform the superimposition
result = cv2.addWeighted(matched_image2, 1, masked_image1, 1, 0)

# Visualize the superimposed image
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Load the UV and visible light images
uv_image = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg")
visible_image = cv2.imread(r"C:\Users\adity\Downloads\img1.jpeg")
# Resize the images to ensure they have the same dimensions
uv_image = cv2.resize(uv_image,None,fx=10,fy=10,interpolation=cv2.INTER_CUBIC)

# Convert the UV image to grayscale
uv_gray = cv2.cvtColor(uv_image, cv2.COLOR_BGR2GRAY)

# Normalize the UV image
uv_normalized = cv2.normalize(uv_gray, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# Create a 3-channel UV image
uv_image_rgb = cv2.cvtColor(uv_normalized, cv2.COLOR_GRAY2BGR)
visible_image = cv2.resize(visible_image,(uv_image_rgb.shape[1],uv_image_rgb.shape[0]))
# Overlay the UV image onto the visible light image
overlay = cv2.addWeighted(visible_image, 0.7, uv_image_rgb, 0.3, 0)
overlay = cv2.resize(overlay,(1280,720),interpolation=cv2.INTER_CUBIC)

# Display the overlaid image
cv2.imshow('Overlay', overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Step 1: UV Image Analysis
uv_image = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg", 0)  # Read the UV image in grayscale

# Preprocessing
uv_image = cv2.medianBlur(uv_image, 1)  # Apply median blur for noise reduction

# Thresholding
_, threshold = cv2.threshold(uv_image, 100, 255, cv2.THRESH_BINARY)  # Adjust the threshold value as needed

# Contour detection
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Step 2: Visible Light Image Localization
visible_image = cv2.imread(r"C:\Users\adity\Downloads\img1.jpeg")

# Convert the visible image to grayscale
gray_visible = cv2.cvtColor(visible_image, cv2.COLOR_BGR2GRAY)

# Create a mask of the UV regions by drawing the contours on a black background
mask = np.zeros_like(gray_visible)
cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)

# Apply the mask to the visible image to extract the UV regions
uv_regions = cv2.bitwise_and(visible_image, visible_image, mask=mask)

# Create a superimposed image by adding the UV regions to the visible image
superimposed_image = cv2.addWeighted(visible_image, 0.7, uv_regions, 0.3, 3)
#
# Display the superimposed image
cv2.imshow('Superimposed Image', uv_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load the two images
image1 = cv2.imread(r"C:\Users\adity\Downloads\img1.jpeg")
image2 = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg")

# Adjust the zoom level of image1
resized_image1 = cv2.resize(image1, None, fx=0.3, fy=0.3)  # Adjust the zoom level as desired

# Get the dimensions of image2
height, width, _ = image1.shape

# Create a new blank image with the dimensions of image2
output_image = np.zeros((height, width, 3), dtype=np.uint8)

# Calculate the translation offset for image2
tx = 90  # Translate along the x-axis (horizontal)
ty = 40  # Translate along the y-axis (vertical)

# Paste image1 onto the blank image
output_image[:resized_image1.shape[0], :resized_image1.shape[1]] = resized_image1

# Superimpose image2 on top of image1 with translation
M = np.float32([[1, 0, tx], [0, 1, ty]])  # Translation matrix
translated_image2 = cv2.warpAffine(resized_image1, M, (width, height))  # Apply translation to image2
blended_image = cv2.addWeighted(output_image, 0.7, translated_image2, 0.3, 0)  # Adjust the blending ratio as desired
# blended_image=cv2.cvtColor(blended_image,cv2.COLOR_GRAY2BGR)
# Display the resulting image
cv2.imshow('Superimposed Image', blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the resulting image
cv2.imwrite('superimposed_image.jpg', blended_image)


In [ ]:
import cv2
import numpy as np

# Load the two images
image1 = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg")
image2 = cv2.imread(r"C:\Users\adity\Downloads\img1.jpeg")

# Adjust the zoom level of image1
zoom_factor = 0.3  # Adjust the zoom level as desired
zoomed_image1 = cv2.resize(image1, None, fx=zoom_factor, fy=zoom_factor)

# Calculate the translation offsets for the zoomed image
tx = 260  # Translate along the x-axis (horizontal)
ty = 300  # Translate along the y-axis (vertical)

# Create a translation matrix for the zoomed image
M = np.float32([[1, 0, tx], [0, 1, ty]])

# Apply translation to the zoomed image
translated_zoomed_image = cv2.warpAffine(zoomed_image1, M, (image2.shape[1], image2.shape[0]))

# Equalize the histograms of the color channels separately
equalized_image1 = cv2.merge([cv2.equalizeHist(image2[:,:,0]), cv2.equalizeHist(image2[:,:,1]), cv2.equalizeHist(image2[:,:,2])])
equalized_translated_zoomed_image = cv2.merge([cv2.equalizeHist(translated_zoomed_image[:,:,0]), cv2.equalizeHist(translated_zoomed_image[:,:,1]), cv2.equalizeHist(translated_zoomed_image[:,:,2])])

# Superimpose the equalized images
superimposed_image = cv2.addWeighted(equalized_image1, 0.85, equalized_translated_zoomed_image, 0.15, 0)  # Adjust the blending ratio as desired

# Equalize the histograms of the color channels in the superimposed image
equalized_superimposed_image = cv2.merge([cv2.equalizeHist(superimposed_image[:, :, 0]), cv2.equalizeHist(superimposed_image[:, :, 1]), cv2.equalizeHist(superimposed_image[:, :, 2])])

# Display the resulting image
cv2.imshow('Superimposed Image', equalized_superimposed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('superimposed_image2.jpg', equalized_superimposed_image)


In [ ]:
import cv2
import numpy as np

# Step 1: Simultaneous UV and visible image capture
uv_camera = cv2.VideoCapture(0)  # UV camera
visible_camera = cv2.VideoCapture(1)  # Visible camera

# Load camera calibration parameters
# These parameters can be obtained by calibrating the cameras using a calibration target
# Adjust the file paths according to your calibration result
calibration_file_uv = "uv_camera_calibration.xml"
calibration_file_visible = "visible_camera_calibration.xml"
calibration_data_uv = cv2.FileStorage(calibration_file_uv, cv2.FILE_STORAGE_READ)
calibration_data_visible = cv2.FileStorage(calibration_file_visible, cv2.FILE_STORAGE_READ)
camera_matrix_uv = calibration_data_uv.getNode("camera_matrix").mat()
dist_coeffs_uv = calibration_data_uv.getNode("distortion_coefficients").mat()
camera_matrix_visible = calibration_data_visible.getNode("camera_matrix").mat()
dist_coeffs_visible = calibration_data_visible.getNode("distortion_coefficients").mat()
calibration_data_uv.release()
calibration_data_visible.release()

# Create a feature detector
detector = cv2.ORB_create()

# Create a feature matcher
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Initialize variables for image registration
prev_kp_uv, prev_des_uv = None, None

while True:
    # Step 2: Preprocessing
    ret_uv, frame_uv = uv_camera.read()
    ret_visible, frame_visible = visible_camera.read()

    # Perform preprocessing on frame_uv and frame_visible if needed
    preprocessed_uv = cv2.cvtColor(frame_uv, cv2.COLOR_BGR2GRAY)
    preprocessed_visible = cv2.cvtColor(frame_visible, cv2.COLOR_BGR2GRAY)

    # Undistort the UV and visible images
    preprocessed_uv = cv2.undistort(preprocessed_uv, camera_matrix_uv, dist_coeffs_uv)
    preprocessed_visible = cv2.undistort(preprocessed_visible, camera_matrix_visible, dist_coeffs_visible)

    # Step 3: Image registration
    # Detect keypoints and compute descriptors for the UV image
    kp_uv, des_uv = detector.detectAndCompute(preprocessed_uv, None)

    if prev_kp_uv is not None and prev_des_uv is not None:
        # Detect keypoints and compute descriptors for the visible image
        kp_visible, des_visible = detector.detectAndCompute(preprocessed_visible, None)

        # Match keypoints between UV and visible images
        matches = matcher.match(prev_des_uv, des_visible)

        # Filter out good matches using a distance threshold
        good_matches = [match for match in matches if match.distance < 100]

        # Extract matched keypoints from UV and visible images
        uv_points = np.float32([prev_kp_uv[match.queryIdx].pt for match in good_matches]).reshape(-1, 1, 2)
        visible_points = np.float32([kp_visible[match.trainIdx].pt for match in good_matches]).reshape(-1, 1, 2)

        # Estimate the transformation matrix using RANSAC
        M, _ = cv2.findHomography(uv_points, visible_points, cv2.RANSAC, 5.0)

        # Warp the UV image to align with the visible image
        registered_uv = cv2.warpPerspective(preprocessed_uv, M, (preprocessed_visible.shape[1], preprocessed_visible.shape[0]))

        # Display the registered UV image
        cv2.imshow("Registered UV Image", registered_uv)

    # Store the current keypoints and descriptors for the next iteration
    prev_kp_uv, prev_des_uv = kp_uv, des_uv

    # Display the preprocessed UV and visible images
    cv2.imshow("Preprocessed UV Image", preprocessed_uv)
    cv2.imshow("Preprocessed Visible Image", preprocessed_visible)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the cameras and close windows
uv_camera.release()
visible_camera.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Step 1: Simultaneous UV and visible image capture
uv_camera = cv2.VideoCapture(0)  # UV camera
visible_camera = cv2.VideoCapture(1)  # Visible camera

# Create a feature detector
detector = cv2.ORB_create()

# Create a feature matcher
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Initialize variables for image registration
prev_kp_uv, prev_des_uv = None, None

while True:
    # Step 2: Preprocessing
    ret_uv, frame_uv = uv_camera.read()
    ret_visible, frame_visible = visible_camera.read()

    # Perform preprocessing on frame_uv and frame_visible if needed
    preprocessed_uv = cv2.cvtColor(frame_uv, cv2.COLOR_BGR2GRAY)
    preprocessed_uv = cv2.GaussianBlur(preprocessed_uv, (3, 3), 0)
    preprocessed_uv = cv2.equalizeHist(preprocessed_uv)  # Contrast enhancement

    preprocessed_visible = cv2.GaussianBlur(frame_visible, (3, 3), 0)
    preprocessed_visible = cv2.cvtColor(preprocessed_visible, cv2.COLOR_BGR2GRAY)
    preprocessed_visible = cv2.equalizeHist(preprocessed_visible)  # Contrast enhancement

    # Illumination correction using adaptive histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    preprocessed_uv = clahe.apply(preprocessed_uv)
    preprocessed_visible = clahe.apply(preprocessed_visible)

    # Region of interest extraction
    roi_mask = np.zeros_like(preprocessed_uv)
    # Define the region of interest coordinates (x, y, width, height) based on your requirements
    roi = (100, 100, 300, 300)
    roi_mask[roi[1]:roi[1] + roi[3], roi[0]:roi[0] + roi[2]] = 255
    preprocessed_uv = cv2.bitwise_and(preprocessed_uv, roi_mask)
    preprocessed_visible = cv2.bitwise_and(preprocessed_visible, roi_mask)

    # Step 3: Image registration
    # Detect keypoints and compute descriptors for the UV image
    kp_uv, des_uv = detector.detectAndCompute(preprocessed_uv, None)

    if prev_kp_uv is not None and prev_des_uv is not None:
        # Detect keypoints and compute descriptors for the visible image
        kp_visible, des_visible = detector.detectAndCompute(preprocessed_visible, None)

        # Match keypoints between UV and visible images
        matches = matcher.match(prev_des_uv, des_visible)

        # Filter out good matches using a distance threshold
        good_matches = [match for match in matches if match.distance < 100]

        # Extract matched keypoints from UV and visible images
        uv_points = np.float32([prev_kp_uv[match.queryIdx].pt for match in good_matches]).reshape(-1, 1, 2)
        visible_points = np.float32([kp_visible[match.trainIdx].pt for match in good_matches]).reshape(-1, 1, 2)

        # Estimate the transformation matrix using RANSAC
        M, _ = cv2.findHomography(uv_points, visible_points, cv2.RANSAC, 5.0)

        # Warp the UV image to align with the visible image
        registered_uv = cv2.warpPerspective(preprocessed_uv, M, (preprocessed_visible.shape[1], preprocessed_visible.shape[0]))

        # Display the registered UV image
        cv2.imshow("Registered UV Image", registered_uv)

    # Store the current keypoints and descriptors for the next iteration
    prev_kp_uv, prev_des_uv = kp_uv, des_uv

    # Display the preprocessed UV and visible images
    cv2.imshow("Preprocessed UV Image", preprocessed_uv)
    cv2.imshow("Preprocessed Visible Image", preprocessed_visible)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the cameras and close windows
uv_camera.release()
visible_camera.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Set the number of inner corners in the calibration pattern
pattern_size = (9, 6)  # Adjust according to your chessboard pattern

# Set the size of the calibration square in millimeters
square_size = 25.0  # Adjust according to your chessboard pattern

# Set the termination criteria for the calibration process
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points based on the calibration pattern
object_points = np.zeros((np.prod(pattern_size), 3), dtype=np.float32)
object_points[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2) * square_size

# Arrays to store object points and image points from calibration images
obj_points = []  # 3D points in real world space
img_points = []  # 2D points in image plane

# Read calibration images and detect calibration pattern
calibration_images = glob.glob('calibration_images/*.jpg')  # Adjust the path to your calibration images
for image_file in calibration_images:
    image = cv2.imread(image_file)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

    if ret:
        obj_points.append(object_points)
        corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        img_points.append(corners)

        # Draw and display the corners
        image = cv2.drawChessboardCorners(image, pattern_size, corners, ret)
        cv2.imshow('Chessboard Corners', image)
        cv2.waitKey(500)  # Display the image for a short duration

cv2.destroyAllWindows()

# Perform camera calibration
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

# Save the calibration parameters to an XML file
calibration_data = cv2.FileStorage('uv_camera_calibration.xml', cv2.FILE_STORAGE_WRITE)
calibration_data.write("camera_matrix", camera_matrix)
calibration_data.write("distortion_coefficients", dist_coeffs)
calibration_data.release()


In [ ]:
import cv2

# Load the two images
image1 = cv2.imread(r"C:\Users\adity\Downloads\img1.jpeg")
image2 = cv2.imread(r"C:\Users\adity\Downloads\img2.jpeg")

# Perform keypoint detection and feature extraction
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(image1, None)
kp2, des2 = orb.detectAndCompute(image2, None)

# Feature matching
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)
top_matches = matches[:10]

# Calculate zoom factor
zoom_factor = 0.3

if len(top_matches) > 0:
    sum_distance = 0.0

    for match in top_matches:
        sum_distance += match.distance

    average_distance = sum_distance / len(top_matches)
    scale_factor = 1
    zoom_factor = .3+ scale_factor * average_distance

# Resize images
new_width1 = int(image1.shape[1] * zoom_factor)
new_height1 = int(image1.shape[0] * zoom_factor)
resized_image1 = cv2.resize(image1, (new_width1, new_height1))

new_width2 = int(image2.shape[1] * zoom_factor)
new_height2 = int(image2.shape[0] * zoom_factor)
resized_image2 = cv2.resize(image2, (new_width2, new_height2))

# Display the results
cv2.imshow('Image 1', resized_image1)
cv2.imshow('Image 2', resized_image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from PIL import Image

def calculate_brightness(image_path):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    grayscale_image = image.convert('L')

    # Calculate the brightness
    histogram = grayscale_image.histogram()
    pixels = sum(histogram)
    brightness = scale = len(histogram)

    for index in range(0, scale):
        ratio = histogram[index] / pixels
        brightness += ratio * (-scale + index)

    # Normalize the brightness value
    brightness /= scale

    return brightness

# Provide the paths to your two images
image_path1 = 'new1.jpg'
image_path2 = 'new2.jpg'
path3 = 'new3.jpg'
path4  = 'new4.jpg'

# Calculate the brightness of the first image
brightness1 = calculate_brightness(image_path1)
br3=calculate_brightness(path3)
br4=calculate_brightness(path4)

# Calculate the brightness of the second image
brightness2 = calculate_brightness(image_path2)

# Print the brightness values
print("Brightness level of image 1:", brightness1)
print("Brightness level of image 2:", brightness2)
print(br3)
print(br4)

    


In [ ]:
from PIL import Image

def calculate_intensity(image_path):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    grayscale_image = image.convert('L')

    # Get the pixel values
    pixels = list(grayscale_image.getdata())

    # Calculate the average intensity
    intensity = sum(pixels) / len(pixels)

    return intensity

# Provide the path to your image
image_path = 'new1.jpg'

img2='new2.jpg'
img3='new3.jpg'
img4='new4.jpg'

# Calculate the intensity level of the image
intensity = calculate_intensity(image_path)
img2=calculate_intensity(img2)
img3=calculate_intensity(img3)
img4=calculate_intensity(img4)
# Print the intensity level
print("Intensity level:", intensity)
print(img2,img3,img4)



In [ ]:
from PIL import Image, ImageDraw

def find_max_intensity_portions(image_path):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    grayscale_image = image.convert('L')

    # Get the dimensions of the image
    width, height = grayscale_image.size

    # Initialize variables for maximum intensity and portions
    max_intensity = 0
    max_intensity_portions = []

    # Iterate over the image pixels
    for y in range(height):
        for x in range(width):
            # Get the intensity of the current pixel
            intensity = grayscale_image.getpixel((x, y))

            # Check if the intensity is greater than the current maximum
            if intensity > max_intensity:
                max_intensity = intensity
                max_intensity_portions = [(x, y)]
            elif intensity == max_intensity:
                max_intensity_portions.append((x, y))

    return max_intensity_portions

# Provide the path to your image
image_path = 'test2.jpg'

# Find the portions with the maximum intensity
max_intensity_portions = find_max_intensity_portions(image_path)

# Open the image
image = Image.open(image_path)

# Create a draw object
draw = ImageDraw.Draw(image)

# Define the radius of the circles
radius = 10

# Draw a circle around each portion with the maximum intensity
for portion in max_intensity_portions:
    x, y = portion
    circle_coords = (x - radius, y - radius, x + radius, y + radius)
    draw.ellipse(circle_coords, outline='red', width=2)

# Save the modified image
image.save('out3.jpg')


In [ ]:
from PIL import Image, ImageDraw

def find_max_intensity_portions(image_path):
    # Open the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    grayscale_image = image.convert('L')

    # Get the dimensions of the image
    width, height = grayscale_image.size

    # Initialize variables for maximum intensity and portions
    max_intensity = 0
    max_intensity_portions = []

    # Iterate over the image pixels
    for y in range(height):
        for x in range(width):
            # Get the intensity of the current pixel
            intensity = grayscale_image.getpixel((x, y))

            # Check if the intensity is greater than the current maximum
            if intensity > max_intensity:
                max_intensity = intensity
                max_intensity_portions = [(x, y)]
            elif intensity == max_intensity:
                max_intensity_portions.append((x, y))

    return max_intensity_portions

# Provide the path to your image
image_path = 'test1.jpg'

# Find the portions with the maximum intensity
max_intensity_portions = find_max_intensity_portions(image_path)

# Open the image
image = Image.open(image_path)

# Create a blank frame image with the same size as the original image
frame_image = Image.new('RGB', image.size, color='white')

# Create a draw object for the frame image
draw_frame = ImageDraw.Draw(frame_image)

# Define the size of the rectangles
rect_width = 200
rect_height = 200

# Draw a rectangle on the frame image for each portion with the maximum intensity
for portion in max_intensity_portions:
    x, y = portion
    x1 = x - (rect_width // 2)
    y1 = y - (rect_height // 2)
    x2 = x1 + rect_width
    y2 = y1 + rect_height
    draw_frame.rectangle((x1, y1, x2, y2), outline='red', width=2)

# Save the frame image with the rectangles
frame_image.save('out3.jpg')


In [ ]:
import cv2
import numpy as np

# Step 1: Load the image
image = cv2.imread('test2.jpg')
image=cv2.resize(image,(720,720))
# Step 2: Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 3: Calculate pixel intensities
intensities = gray.flatten()

# Step 4: Find the maximum intensity
max_intensity = np.max(intensities)
max_index = np.argmax(intensities)

# Step 5: Highlight or extract the most intense portion
# In this example, we'll draw a rectangle around the portion
# with the maximum intensity.
height, width = gray.shape
max_row = max_index // width
max_col = max_index % width

rect_size = 80  # Size of the rectangle to highlight the portion
top_left = (max_col - rect_size, max_row - rect_size)
bottom_right = (max_col + rect_size, max_row + rect_size)
cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
# Step 6: Display or save the result
cv2.imshow('Most Intense Portion', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Step 1: Load the image
image = cv2.imread('test2.jpg')
image=cv2.resize(image,(720,720))

# Step 2: Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 3: Calculate pixel intensities
intensities = gray.flatten()

# Step 4: Find the maximum intensity
max_intensity = np.max(intensities)
max_indices = np.where(intensities == max_intensity)[0]

# Step 5: Extract the most intense portion
rows, cols = gray.shape
top_row = np.min(max_indices) // cols
bottom_row = np.max(max_indices) // cols
left_col = np.min(max_indices) % cols
right_col = np.max(max_indices) % cols

# Crop the portion with the highest intensity
portion = image[top_row:bottom_row, left_col:right_col]

# Step 6: Draw a rectangle around the portion
cv2.rectangle(image, (left_col, top_row), (right_col, bottom_row), (0, 255, 0), 2)

# Step 7: Display or save the result
cv2.imshow('Most Intense Portion', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Step 1: Load the image
image = cv2.imread('test5.jpg')
image=cv2.resize(image,(720,720))
# Step 2: Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 3: Calculate pixel intensities
intensities = gray.flatten()

# Step 4: Find the maximum intensity
max_intensity = np.max(intensities)
max_indices = np.where(intensities == max_intensity)[0]

# Step 5: Extract the most intense portion
rows, cols = gray.shape
top_row = np.min(max_indices) // cols
bottom_row = np.max(max_indices) // cols
left_col = np.min(max_indices) % cols
right_col = np.max(max_indices) % cols

# Adjust the portion to include the entire connected component
ret, threshold = cv2.threshold(gray, max_intensity - 1, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
max_contour = max(contours, key=cv2.contourArea)
x, y, width, height = cv2.boundingRect(max_contour)
top_row = y
bottom_row = y + height
left_col = x
right_col = x + width

# Crop the portion with the highest intensity
portion = image[top_row:bottom_row, left_col:right_col]

# Step 6: Draw a rectangle around the portion
cv2.rectangle(image, (left_col, top_row), (right_col, bottom_row), (0, 255, 0), 2)

# Step 7: Display or save the result
cv2.imshow('Most Intense Portion', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw


# Step 1: Load the image
image = cv2.imread('test1.jpg')
image=cv2.resize(image,(720,720))
# Step 2: Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 3: Calculate pixel intensities
intensities = gray.flatten()

# Step 4: Find the maximum intensity
max_intensity = np.max(intensities)
max_indices = np.where(intensities == max_intensity)[0]

# Step 5: Extract the most intense portion
rows, cols = gray.shape
top_row = np.min(max_indices) // cols
bottom_row = np.max(max_indices) // cols
left_col = np.min(max_indices) % cols
right_col = np.max(max_indices) % cols

# Adjust the portion to include the entire connected component
ret, threshold = cv2.threshold(gray, max_intensity - 1, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
max_contour = max(contours, key=cv2.contourArea)
x, y, width, height = cv2.boundingRect(max_contour)
top_row = y
bottom_row = y + height
left_col = x
right_col = x + width

# Crop the portion with the highest intensity
portion = image[top_row:bottom_row, left_col:right_col]

# Step 6: Draw a rectangle around the portion
cv2.rectangle(image, (left_col, top_row), (right_col, bottom_row), (0, 255, 0), 2)

# Step 7: Display or save the result
cv2.imshow('Most Intense Portion', image)
# image.save('out1.jpg')
# image.save('out3.jpg')
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Step 1: Load the image
image = cv2.imread('test2.jpg')
image=cv2.resize(image,(1000,1000))
# Step 2: Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 3: Calculate pixel intensities
intensities = gray.flatten()

# Step 4: Find the maximum intensity
max_intensity = np.max(intensities)
max_indices = np.where(intensities == max_intensity)[0]

# Step 5: Extract the most intense portion
rows, cols = gray.shape
top_row = np.min(max_indices) // cols
bottom_row = np.max(max_indices) // cols
left_col = np.min(max_indices) % cols
right_col = np.max(max_indices) % cols

# Crop the portion with the highest intensity
portion = image[top_row:bottom_row, left_col:right_col]

# Step 6: Apply edge detection to the portion
portion_gray = cv2.cvtColor(portion, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(portion_gray, 1000, 1000)

# Step 7: Draw a rectangle around the portion
cv2.rectangle(image, (left_col, top_row), (right_col, bottom_row), (0, 255, 0), 2)

# Step 8: Display or save the result
cv2.imshow('Highest Intensity Portion with Edges', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
